# Time-optimal path planning
This notebook handles standard code for the computation of time-optimal paths in windfields

In [1]:
import os
import sys
import time
import numpy as np
from math import atan, cos, sin
sys.path.extend(['/home/bastien/Documents/work/mermoz', 
                 '/home/bastien/Documents/work/mermoz/src',
                 '/home/bastien/Documents/work/mdisplay',
                 '/home/bastien/Documents/work/mdisplay/src'])

from mermoz.feedback import TargetFB
from mermoz.mdf_manager import MDFmanager
from mermoz.params_summary import ParamsSummary
from mermoz.problem import MermozProblem
from mermoz.model import ZermeloGeneralModel
from mermoz.solver import Solver
from mermoz.stoppingcond import TimedSC, DistanceSC
from mermoz.wind import DiscreteWind
from mermoz.misc import *
from mdisplay.geodata import GeoData

from solver_utils import select_wind

### Problem definition

Initial point, target, UAV airspeed

In [2]:
# Scale factor
sf = 1e6

# Initial point
x_init = sf * np.array((0., 0.))

# Target
x_target = sf * np.array((1., 0.))

# Windfield boundaries
bl = sf * np.array((-0.1, -1.))
tr = sf * np.array((1.1, 1.))

# UAV airspeed in m/s
v_a = 23.

nx_rft = 101
ny_rft = 101

case_name = 'rad_gauss_1'

Set up the problem

In [3]:
# Prepare output directory
output_dir = f'/home/bastien/Documents/work/mermoz/output/example_solver_{case_name}'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Create a file manager to dump problem data
mdfm = MDFmanager()
mdfm.set_output_dir(output_dir)

# Problem coordinates
# coords = COORD_GCS
coords = COORD_CARTESIAN

# Problem wind data
from mermoz.wind import RadialGaussWind
total_wind = RadialGaussWind(sf * 0.5, sf * 0.05, sf * 0.2, 1/3 * np.log(0.5), v_a * 1.1)
def domain(x):
    return bl[0] < x[0] < tr[0] and bl[1] < x[1] < tr[1]

# Time window upper bound
# Estimated through great circle distance + 20 percent
T = 30. * 3600 # 1.2 * geodesic_distance(x_init, x_target, mode='rad') / v_a

# Creates the cinematic model
zermelo_model = ZermeloGeneralModel(v_a, coords=coords)
zermelo_model.update_wind(total_wind)

# Creates the navigation problem on top of the previous model
mp = MermozProblem(zermelo_model, T=T, coords=coords, autodomain=False, domain=domain)

### Solver parameters

In [8]:
auto_psi = 0.
psi_min = - DEG_TO_RAD * 90. #auto_psi - DEG_TO_RAD * 20.
psi_max = DEG_TO_RAD * 90. #auto_psi + DEG_TO_RAD * 20.

nt_pmp = 1000
opti_ceil = 0.01 * sf
neighb_ceil = opti_ceil/2.

solver = Solver(mp,
                x_init,
                x_target,
                T,
                psi_min,
                psi_max,
                output_dir,
                N_disc_init=2,
                opti_ceil=opti_ceil,
                neighb_ceil=neighb_ceil,
                n_min_opti=1,
                adaptive_int_step=False,
                N_iter=nt_pmp)

solver.log_config()
solver.setup()

Solve problem

In [9]:
t_start = time.time()
solver.solve_fancy()
t_end = time.time()
time_pmp = t_end - t_start

Shooting -90.0
Shooting 90.0
1000000.00000, 0.00
Iteration 1 - Shooting 0.0 - Depth 1 - Branching LU
Iteration 2 - Shooting -45.0 - Depth 2 - Branching L
Iteration 3 - Shooting -67.5 - Depth 3 - Branching LU
Iteration 4 - Shooting -56.2 - Depth 4 - Branching LU
Iteration 5 - Shooting -50.6 - Depth 5 - Branching LU
Iteration 6 - Shooting -47.8 - Depth 6 - Branching LU
Iteration 7 - Shooting -46.4 - Depth 7 - Branching LU
Iteration 8 - Shooting -45.7 - Depth 8 - Branching LU
Iteration 9 - Shooting -45.4 - Depth 9 - Branching LU
Iteration 10 - Shooting -45.2 - Depth 10 - Max depth (10) reached, no branching
Iteration 11 - Shooting -45.5 - Depth 10 - Max depth (10) reached, no branching
Iteration 12 - Shooting -46.1 - Depth 9 - Branching LU
Iteration 13 - Shooting -45.9 - Depth 10 - Max depth (10) reached, no branching
Iteration 14 - Shooting -46.2 - Depth 10 - Max depth (10) reached, no branching
Iteration 15 - Shooting -47.1 - Depth 8 - Branching LU
Iteration 16 - Shooting -46.8 - Depth 

Iteration 125 - Shooting 34.5 - Depth 8 - Branching LU
Iteration 126 - Shooting 34.8 - Depth 9 - No branching
Iteration 127 - Shooting 34.1 - Depth 9 - No branching
Iteration 128 - Shooting 28.1 - Depth 5 - Branching LU
Iteration 129 - Shooting 30.9 - Depth 6 - Branching LU
Iteration 130 - Shooting 32.3 - Depth 7 - Branching LU
Iteration 131 - Shooting 33.0 - Depth 8 - Branching LU
Iteration 132 - Shooting 33.4 - Depth 9 - No branching
Iteration 133 - Shooting 32.7 - Depth 9 - No branching
Iteration 134 - Shooting 31.6 - Depth 8 - Branching LU
Iteration 135 - Shooting 32.0 - Depth 9 - No branching
Iteration 136 - Shooting 31.3 - Depth 9 - No branching
Iteration 137 - Shooting 29.5 - Depth 7 - Branching LU
Iteration 138 - Shooting 30.2 - Depth 8 - Branching LU
Iteration 139 - Shooting 30.6 - Depth 9 - No branching
Iteration 140 - Shooting 29.9 - Depth 9 - No branching
Iteration 141 - Shooting 28.8 - Depth 8 - Branching LU
Iteration 142 - Shooting 29.2 - Depth 9 - No branching
Iteration 

Iteration 263 - Shooting 58.0 - Depth 9 - Branching LU
Iteration 264 - Shooting 57.8 - Depth 10 - Max depth (10) reached, no branching
Iteration 265 - Shooting 58.2 - Depth 10 - Max depth (10) reached, no branching
Iteration 266 - Shooting 58.7 - Depth 9 - Branching LU
Iteration 267 - Shooting 58.5 - Depth 10 - Max depth (10) reached, no branching
Iteration 268 - Shooting 58.9 - Depth 10 - Max depth (10) reached, no branching
Iteration 269 - Shooting 60.5 - Depth 7 - Branching LU
Iteration 270 - Shooting 59.8 - Depth 8 - Branching LU
Iteration 271 - Shooting 59.4 - Depth 9 - Branching LU
Iteration 272 - Shooting 59.2 - Depth 10 - Max depth (10) reached, no branching
Iteration 273 - Shooting 59.6 - Depth 10 - Max depth (10) reached, no branching
Iteration 274 - Shooting 60.1 - Depth 9 - Branching LU
Iteration 275 - Shooting 59.9 - Depth 10 - Max depth (10) reached, no branching
Iteration 276 - Shooting 60.3 - Depth 10 - Max depth (10) reached, no branching
Iteration 277 - Shooting 61.2 

Iteration 389 - Shooting 69.6 - Depth 8 - Branching LU
Iteration 390 - Shooting 69.3 - Depth 9 - Branching LU
Iteration 391 - Shooting 69.1 - Depth 10 - Max depth (10) reached, no branching
Iteration 392 - Shooting 69.4 - Depth 10 - Max depth (10) reached, no branching
Iteration 393 - Shooting 70.0 - Depth 9 - Branching LU
Iteration 394 - Shooting 69.8 - Depth 10 - Max depth (10) reached, no branching
Iteration 395 - Shooting 70.1 - Depth 10 - Max depth (10) reached, no branching
Iteration 396 - Shooting 71.7 - Depth 7 - Branching LU
Iteration 397 - Shooting 71.0 - Depth 8 - Branching LU
Iteration 398 - Shooting 70.7 - Depth 9 - Branching LU
Iteration 399 - Shooting 70.5 - Depth 10 - Max depth (10) reached, no branching
Iteration 400 - Shooting 70.8 - Depth 10 - Max depth (10) reached, no branching
Iteration 401 - Shooting 71.4 - Depth 9 - Branching LU
Iteration 402 - Shooting 71.2 - Depth 10 - Max depth (10) reached, no branching
Iteration 403 - Shooting 71.5 - Depth 10 - Max depth (1

Iteration 509 - Shooting 79.5 - Depth 8 - Branching LU
Iteration 510 - Shooting 79.1 - Depth 9 - Branching LU
Iteration 511 - Shooting 78.9 - Depth 10 - Max depth (10) reached, no branching
Iteration 512 - Shooting 79.3 - Depth 10 - Max depth (10) reached, no branching
Iteration 513 - Shooting 79.8 - Depth 9 - Branching LU
Iteration 514 - Shooting 79.6 - Depth 10 - Max depth (10) reached, no branching
Iteration 515 - Shooting 80.0 - Depth 10 - Max depth (10) reached, no branching
Iteration 516 - Shooting 80.9 - Depth 8 - Branching LU
Iteration 517 - Shooting 80.5 - Depth 9 - Branching LU
Iteration 518 - Shooting 80.3 - Depth 10 - Max depth (10) reached, no branching
Iteration 519 - Shooting 80.7 - Depth 10 - Max depth (10) reached, no branching
Iteration 520 - Shooting 81.2 - Depth 9 - Branching LU
Iteration 521 - Shooting 81.0 - Depth 10 - Max depth (10) reached, no branching
Iteration 522 - Shooting 81.4 - Depth 10 - Max depth (10) reached, no branching
Iteration 523 - Shooting -84.4

### Explicit control laws

In [6]:
sc = DistanceSC(lambda x: np.linalg.norm(x - x_target), opti_ceil)
sc = TimedSC(T)
from mermoz.feedback import FixedHeadingFB, ConstantFB

mp.load_feedback(FixedHeadingFB(mp.model.wind, v_a, 0., mp.coords))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

mp.load_feedback(ConstantFB(0.))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

### RFT

In [7]:
from mermoz.rft import RFT
t_start = time.time()
rft = RFT(bl, tr, T, nx_rft, ny_rft, mp, x_init, kernel='matlab', coords=COORD_CARTESIAN)
rft.compute()
t_end = time.time()
time_rft = t_end - t_start


tol =

       12000

	151 steps in 3.72 seconds from 0 to 13500
	151 steps in 4.07 seconds from 13500 to 27000
	151 steps in 2.72 seconds from 27000 to 40500
	151 steps in 3.91 seconds from 40500 to 54000
	151 steps in 2.84 seconds from 54000 to 67500
	151 steps in 2.68 seconds from 67500 to 81000
	151 steps in 3 seconds from 81000 to 94500
	151 steps in 2.86 seconds from 94500 to 108000

Total execution time 26.34 seconds


Dump everything to output directory

In [10]:
nx = 1001
ny = 1001
mdfm.dump_wind(total_wind, nx=101, ny=101, bl=bl, tr=tr)
mdfm.dump_trajs(mp.trajs)
# rft.dump_rff(output_dir)


ps = ParamsSummary({}, output_dir)
ps.load_from_solver(solver)
ps.add_param('bl_wind', tuple(bl))
ps.add_param('tr_wind', tuple(tr))
#ps.add_param('nx_wind', nx)
#ps.add_param('ny_wind', ny)
ps.add_param('nt_pmp', nt_pmp)
ps.add_param('pmp_time', time_pmp)
ps.dump()